## CBoW modelio mokymas

Šis užrašų knygelė yra dalis [AI for Beginners Curriculum](http://aka.ms/ai-beginners)

Šiame pavyzdyje apžvelgsime, kaip apmokyti CBoW kalbos modelį, kad sukurtume savo Word2Vec įterpimo erdvę. Naudosime AG News duomenų rinkinį kaip teksto šaltinį.


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Pirmiausia įkelkime savo duomenų rinkinį ir apibrėžkime žodžių skirstytuvą bei žodyną. Nustatysime `vocab_size` reikšmę į 5000, kad šiek tiek apribotume skaičiavimus.


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW Modelis

CBoW modelis mokosi numatyti žodį remdamasis $2N$ kaimyniniais žodžiais. Pavyzdžiui, kai $N=1$, iš sakinio *I like to train networks* gausime šias poras: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks). Čia pirmasis žodis yra kaimyninis žodis, naudojamas kaip įvestis, o antrasis žodis yra tas, kurį prognozuojame.

Norint sukurti tinklą, kuris prognozuotų kitą žodį, mums reikės pateikti kaimyninį žodį kaip įvestį ir gauti žodžio numerį kaip išvestį. CBoW tinklo architektūra yra tokia:

* Įvesties žodis perduodamas per įterpimo sluoksnį. Šis įterpimo sluoksnis bus mūsų Word2Vec įterpimas, todėl jį apibrėšime atskirai kaip kintamąjį `embedder`. Šiame pavyzdyje naudosime įterpimo dydį = 30, nors galite eksperimentuoti su didesniais matmenimis (tikrasis Word2Vec turi 300).
* Įterpimo vektorius tada perduodamas per linijinį sluoksnį, kuris prognozuos išvesties žodį. Todėl jis turi `vocab_size` neuronų.

Kalbant apie išvestį, jei naudosime `CrossEntropyLoss` kaip nuostolių funkciją, taip pat turėsime pateikti tik žodžių numerius kaip laukiamus rezultatus, be vieno karšto kodavimo (one-hot encoding).


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## Mokymo duomenų paruošimas

Dabar suprogramuokime pagrindinę funkciją, kuri apskaičiuos CBoW žodžių poras iš teksto. Ši funkcija leis mums nurodyti lango dydį ir grąžins porų rinkinį - įvesties ir išvesties žodį. Atkreipkite dėmesį, kad ši funkcija gali būti naudojama tiek su žodžiais, tiek su vektoriais/tensoriais - tai leis mums užkoduoti tekstą prieš perduodant jį funkcijai `to_cbow`.


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


Pasiruoškime mokymo duomenų rinkinį. Peržiūrėsime visas naujienas, iškviesime `to_cbow`, kad gautume žodžių porų sąrašą, ir pridėsime tas poras į `X` ir `Y`. Siekdami sutaupyti laiko, apsiribosime pirmomis 10 tūkst. naujienų - jei turite daugiau laiko laukti ir norite gauti geresnius įterpimus, lengvai galite pašalinti šį apribojimą :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

Mes taip pat konvertuosime tuos duomenis į vieną duomenų rinkinį ir sukursime duomenų įkroviklį:


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

Mes taip pat konvertuosime tuos duomenis į vieną duomenų rinkinį ir sukursime duomenų įkroviklį:


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

Dabar pradėkime tikrąjį mokymą. Naudosime `SGD` optimizatorių su gana dideliu mokymosi greičiu. Taip pat galite pabandyti naudoti kitus optimizatorius, tokius kaip `Adam`. Iš pradžių treniruosime 10 epochų – ir, jei norite dar mažesnio nuostolio, galite vėl paleisti šią langelį.


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Bandymas su Word2Vec

Norėdami naudoti Word2Vec, ištraukime vektorius, atitinkančius visus žodžius mūsų žodyne:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

Pažiūrėkime, pavyzdžiui, kaip žodis **Paryžius** yra užkoduotas į vektorių:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


Įdomu naudoti Word2Vec ieškant sinonimų. Ši funkcija grąžins `n` artimiausių žodžių pagal pateiktą įvestį. Norėdami juos rasti, apskaičiuojame $|w_i - v|$ normą, kur $v$ yra mūsų įvesties žodžiui atitinkantis vektorius, o $w_i$ yra $i$-tojo žodžio žodyne kodavimas. Tada surūšiuojame masyvą ir naudojame `argsort`, kad gautume atitinkamus indeksus, bei paimame pirmus `n` sąrašo elementus, kurie nurodo artimiausių žodžių pozicijas žodyne.


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## Pagrindinė mintis

Naudodami sumanius metodus, tokius kaip CBoW, galime apmokyti Word2Vec modelį. Taip pat galite pabandyti apmokyti skip-gram modelį, kuris yra mokomas numatyti kaimyninį žodį, turint centrinį, ir pažiūrėti, kaip gerai jis veikia.



---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors stengiamės užtikrinti tikslumą, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama naudoti profesionalų žmogaus vertimą. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius dėl šio vertimo naudojimo.
